In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark import SparkContext, SparkConf

In [6]:
file = "home/steam/Game_Reviews.csv"

# Pre-process reviews with Pandas

In [8]:
Game_Reviews = pd.read_csv(file, header = [0] )

/opt/conda/anaconda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
review_vote = Game_Reviews[["review","voted_up"]]

In [10]:
review_vote['review'] = review_vote['review'].astype(str)
review_vote['review'] = review_vote['review'].apply(lambda x: x.replace("\n", " "))
review_vote['vote'] = review_vote['voted_up'].apply(lambda x: 1 if x is True else 0)

/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [11]:
# Remove the punctuations
import string
import re

remove_chars = '[·’!"\#$%&\'()＃！*+,-./:;<=>?\@，：?￥★、…．＞【】［］《》？“”‘’\[\\]^_`{|}~]+'

review_vote['review'] = review_vote['review'].apply(lambda x: re.sub(remove_chars, "", x))

/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [12]:
review_vote = review_vote[["review","vote"]]
review_vote

,review,vote
0,So if you are like me and you want to play as ...,1
1,You can play as General Grievous You can kill ...,1
2,Yoda has a fat a in this game can confirm,1
3,You can kill younglings Best Game GOTY 1010,1
4,this game is what i imagine sex is like,1
...,...,...
11287435,big man on horse very scary bro,1
11287436,this is a dark soul,1
11287437,Dog,1
11287438,if i have to tell you why this is a good game ...,1


In [13]:
# Remove redundant spaces
review_vote["review"] = review_vote["review"].replace('\s+', ' ', regex=True)

In [14]:
# drop na
review_vote = review_vote[review_vote["review"] != '']

In [15]:
# save to a .csv file
review_data = review_vote.to_csv("/home/steam/review_data.csv")

# Modeling with Pyspark

In [2]:
file2 = "hdfs://cluster-steam-m/user/dataproc/review_data.csv"

In [3]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import NGram
from pyspark.ml import Pipeline

In [4]:
sc=SparkContext(master="local[8]")
spark = SparkSession.builder.appName('wk7_lr').getOrCreate()

In [5]:
review_info = spark.read.csv(file2, inferSchema = True, header =True)
review_info.show(5, truncate = False)

+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
review_info.count()

11230425

In [6]:
review_info = review_info.dropna().select("review","vote")

In [ ]:
tokenizer = Tokenizer(inputCol="review", outputCol="words")
wordsData = tokenizer.transform(review_info)
wordsData.show(5, truncate=False)

remover = StopWordsRemover(inputCol="words", outputCol="filtered")
filtData = remover.transform(wordsData)
filtData.show(5, truncate=False)

hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(filtData)
# alternatively, CountVectorizer can also be used to get term frequency vectors
featurizedData.show(5, truncate=False)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("vote", "features").show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
rescaledData = rescaledData.withColumnRenamed("vote", "label").filter(size("words") > 1)

In [ ]:
rescaledData.show(5)

+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|              review|label|               words|            filtered|         rawFeatures|            features|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|So if you are lik...|    1|[so, if, you, are...|[like, want, play...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
|You can play as G...|    1|[you, can, play, ...|[play, general, g...|(20,[0,2,4,5,6,7,...|(20,[0,2,4,5,6,7,...|
|Yoda has a fat a ...|    1|[yoda, has, a, fa...|[yoda, fat, game,...|(20,[3,11,15,19],...|(20,[3,11,15,19],...|
|You can kill youn...|    1|[you, can, kill, ...|[kill, younglings...|(20,[2,3,6,14,15,...|(20,[2,3,6,14,15,...|
|this game is what...|    1|[this, game, is, ...|[game, imagine, s...|(20,[0,3,10,15],[...|(20,[0,3,10,15],[...|
+--------------------+-----+--------------------+--------------------+--------------------+-----

In [13]:
# split dataset into 0.8/0.2 (train/val/test)
# train_set, test_set = rescaledData.randomSplit([0.8, 0.2], seed = 336)
zeros = rescaledData.filter(rescaledData["label"]==0)
ones = rescaledData.filter(rescaledData["label"]==1)
# split datasets into training and testing
train0, test0 = zeros.randomSplit([0.8,0.2], seed=1234)
train1, test1 = ones.randomSplit([0.8,0.2], seed=1234)
# stack datasets back together
train_set = train0.union(train1)
test_set = test0.union(test1)

In [14]:
train_set.show(5)

+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|              review|label|               words|            filtered|         rawFeatures|            features|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|    1060 No 1060 Yes|    0|[, 1060, no, 1060...| [, 1060, 1060, yes]|(20,[1,12,15],[2....|(20,[1,12,15],[1....|
| 14082013 You can...|    0|[, 14082013, you,...|[, 14082013, buy,...|(20,[0,1,2,4,5,6,...|(20,[0,1,2,4,5,6,...|
| 5 PM game isnt o...|    0|[, 5, pm, game, i...|[, 5, pm, game, i...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
| 6 years of devel...|    0|[, 6, years, of, ...|[, 6, years, deve...|(20,[0,2,5,6,8,12...|(20,[0,2,5,6,8,12...|
| 6 years since CS...|    0|[, 6, years, sinc...|[, 6, years, sinc...|(20,[0,1,2,4,5,6,...|(20,[0,1,2,4,5,6,...|
+--------------------+-----+--------------------+--------------------+--------------------+-----

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=100)
lrModel = lr.fit(train_set)
predictions = lrModel.transform(test_set)

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

In [ ]:
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(test_set.count())
accuracy

## Improve with Ngram method

In [ ]:
# tokenizer = Tokenizer(inputCol="review", outputCol="words")
# wordsData = tokenizer.transform(review_info)
# wordsData.show(5, truncate=False)

# remover = StopWordsRemover(inputCol="words", outputCol="filtered")
# filtData = remover.transform(wordsData)
# filtData.show(5, truncate=False)

# ngram = NGram(n=2, inputCol="filtered", outputCol="ngrams")
# ngramData = ngram.transform(filtData)

# hashingTF = HashingTF(inputCol="ngrams", outputCol="rawFeatures", numFeatures=20)
# featurizedData = hashingTF.transform(ngramData)
# # alternatively, CountVectorizer can also be used to get term frequency vectors
# featurizedData.show(5, truncate=False)

# idf = IDF(inputCol="rawFeatures", outputCol="features")
# idfModel = idf.fit(featurizedData)
# rescaledData = idfModel.transform(featurizedData)

In [7]:
# Write in the pipeline structure

tokenizer = Tokenizer(inputCol="review", outputCol="words")

remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered")

ngram = NGram(n=2, inputCol=remover.getOutputCol(), outputCol="ngrams")

hashingTF = HashingTF(inputCol=ngram.getOutputCol(), outputCol="rawFeatures", numFeatures=20)

idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features")

pipeline = Pipeline(stages=[tokenizer, remover, ngram, hashingTF, idf])

In [8]:
review_info = review_info.na.drop(subset=["review"])

In [9]:
review_info = review_info.withColumnRenamed("vote", "label")

In [10]:
pipelineFit = pipeline.fit(review_info)
pipe_data = pipelineFit.transform(review_info)
pipe_data = pipe_data.filter(size("words")>1).filter(size("ngrams")>1).select("label","features")
pipe_data = pipe_data.where(col("features").isNotNull()) 
pipe_data = pipe_data.where(col("label").isNotNull()) 

In [12]:
# stratify the randomsplit
zeros = pipe_data.filter(pipe_data["label"]==0)
ones = pipe_data.filter(pipe_data["label"]==1)
# split datasets into training and testing
train0, test0 = zeros.randomSplit([0.8,0.2], seed=1234)
train1, test1 = ones.randomSplit([0.8,0.2], seed=1234)
# stack datasets back together
train = train0.union(train1)
test = test0.union(test1)

In [ ]:
lr2 = LogisticRegression(maxIter=500)
lr_model = lr2.fit(train.where(col("features").isNotNull()))
predictions = lr_model.transform(test)

In [ ]:
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(test.count())
roc_auc = evaluator.evaluate(predictions)

0.8198213392033603

In [30]:
accuracy

0.8198213392033603

In [33]:
predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|(20,[0],[34.85728...|[-0.0757906155245...|[0.48106141086549...|       1.0|
|    0|(20,[0,1],[0.9959...|[-1.1659609178260...|[0.23758584317283...|       1.0|
|    0|(20,[0,1],[0.9959...|[-1.1659609178260...|[0.23758584317283...|       1.0|
|    0|(20,[0,1,2,3,4,5,...|[-1.1342934797478...|[0.24336962503038...|       1.0|
|    0|(20,[0,1,2,3,4,5,...|[-1.0361236345733...|[0.26189863344517...|       1.0|
|    0|(20,[0,1,2,3,4,5,...|[-1.0625662166428...|[0.25681935286546...|       1.0|
|    0|(20,[0,1,2,3,4,5,...|[-1.3015058373132...|[0.21391169608596...|       1.0|
|    0|(20,[0,1,2,3,4,5,...|[-1.0004315939004...|[0.26885657332151...|       1.0|
|    0|(20,[0,1,2,3,4,5,...|[-1.2114296461323...|[0.22944818929932...|       1.0|
|    0|(20,[0,1,

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics

my_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_lr.evaluate(predictions)

0.7424723232521575

In [15]:
print(5)

5


In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Select (prediction, true label) and compute test error
f1_evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol="prediction", metricName="f1")

In [ ]:
# other methods
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

dtc = DecisionTreeClassifier(labelCol='label',featuresCol='features')
dtc_model = dtc.fit(train)
dtc_predictions = dtc_model.transform(test)
dtc_f1 = f1_evaluator.evaluate(dtc_predictions)
print('A single decision tree had a f1 of: {0:2.2f}%'.format(dtc_f1*100))

A single decision tree had a f1 of: 75.13%


In [ ]:
# Train the models (its three models, so it might take some time)
rfc = RandomForestClassifier(labelCol='label',featuresCol='features')
rfc_model = rfc.fit(train)
rfc_predictions = rfc_model.transform(test)
rfc_f1 = f1_evaluator.evaluate(rfc_predictions)
print('A random forest ensemble had a f1 of: {0:2.2f}%'.format(rfc_f1*100))

A random forest ensemble had a f1 of: 75.10%


In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

gbt = GBTClassifier(labelCol='label',featuresCol='features')
gbt_model = gbt.fit(train)
gbt_predictions = gbt_model.transform(test)
gbt_f1 = f1_evaluator.evaluate(gbt_predictions)
print('A ensemble using GBT hada f1 of: {0:2.2f}%'.format(gbt_f1*100))

A ensemble using GBT hada f1 of: 75.35%
